## Introduction
DataSource is a helper class for doing montonous, repetitive tasks related to financial time series data. Quite often, we need to apply a set of common functions such as shifting data, calculating rolling statistics, percent changes etc., for each security in our database by using pandas **groupby** function. DataSource simplifies this process by one-liners instead of multiple lines of pandas code; In fact, its just a fancy wrapper for pandas groupby with some extra goodies.

So instead of 
```python
shift = lambda x: x.shift(1)
dataframe['lag_one'] = dataframe.groupby('symbol')['close'].transform(shift)
```
it would be
```python
dataframe.add_lag(on='close', period=1, col_name='lag_one')
```

The only requirement is that the dataframe must have **symbol and timestamp columns**. 

**If you have them as indexes, reset them as columns**

In [1]:
import pandas as pd
from fastbt.datasource import DataSource

## Initialize DataSource class with a dataframe

In [2]:
df = pd.read_csv('data/bank.csv', parse_dates=['timestamp'])
ds = DataSource(df)

## Use ds.data to get the dataframe back
ds.data.head()

,timestamp,symbol,series,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,isin
0,2018-09-03,AXISBANK,EQ,655.45,655.45,629.6,631.8,630.70,649.25,6494484,4.148248e+09,88364,INE238A01034
1,2018-09-03,RBLBANK,EQ,630.10,643.50,622.0,623.9,624.40,627.25,1532438,9.708929e+08,28755,INE976G01028
2,2018-09-03,INDUSINDBK,EQ,1906.00,1918.85,1884.7,1897.0,1895.00,1906.60,937981,1.782402e+09,85683,INE095A01012
3,2018-09-03,KOTAKBANK,EQ,1292.00,1295.00,1265.0,1269.1,1266.25,1287.25,2076747,2.639495e+09,59594,INE237A01028
4,2018-09-03,BANKBARODA,EQ,153.95,156.50,150.9,151.7,151.30,152.95,16081701,2.484263e+09,72339,INE028A01039


In [4]:
# If your dataframe has a different name for the symbol and timestamp
# column, pass them as parameters during initialization
ds = DataSource(df, symbol='symbol', timestamp='timestamp')

DataSource adds a column for each function you specify and returns a dataframe with the column added. The existing functions are

```
 add_lag
 add_pct_change
 add_rolling
 add_formula
 add_indicator 
```

All functions have a col_name argument. Except for ``add_formula``, column names are generated automatically for all functions. 

**All column names, even those specified as arguments, are converted into lower case to make them case-insensitive.**

The following arguments are common to the functions
 * ``col_name`` - column name to be added in dataframe. Mandatory for ``add_formula``, for others added automatically
 * ``period`` - time period for the function
 * ``lag`` - time lag; time by which the result is to be lagged. Not applicable for ``add_lag`` and ``add_formula``
 * ``on`` - column on which the grouping is to be made. Not applicable to ``add_formula`` and ``add_indicator``
 
 Let's see a few examples

## ``add_lag``

adds the given time lag to the specified period

In [5]:
## Add a one day lag to data
ds.add_lag();
print(ds.data.info()) # Column lag_close_1 added automatically
ds.data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 14 columns):
timestamp      48 non-null datetime64[ns]
symbol         48 non-null object
series         48 non-null object
open           48 non-null float64
high           48 non-null float64
low            48 non-null float64
close          48 non-null float64
last           48 non-null float64
prevclose      48 non-null float64
tottrdqty      48 non-null int64
tottrdval      48 non-null float64
totaltrades    48 non-null int64
isin           48 non-null object
lag_close_1    36 non-null float64
dtypes: datetime64[ns](1), float64(8), int64(2), object(3)
memory usage: 5.3+ KB
None


,timestamp,symbol,series,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,isin,lag_close_1
0,2018-09-03,AXISBANK,EQ,655.45,655.45,629.6,631.8,630.70,649.25,6494484,4.148248e+09,88364,INE238A01034,NaN
1,2018-09-03,RBLBANK,EQ,630.10,643.50,622.0,623.9,624.40,627.25,1532438,9.708929e+08,28755,INE976G01028,NaN
2,2018-09-03,INDUSINDBK,EQ,1906.00,1918.85,1884.7,1897.0,1895.00,1906.60,937981,1.782402e+09,85683,INE095A01012,NaN
3,2018-09-03,KOTAKBANK,EQ,1292.00,1295.00,1265.0,1269.1,1266.25,1287.25,2076747,2.639495e+09,59594,INE237A01028,NaN
4,2018-09-03,BANKBARODA,EQ,153.95,156.50,150.9,151.7,151.30,152.95,16081701,2.484263e+09,72339,INE028A01039,NaN


In [12]:
# Add a 2 day lag
ds.add_lag(period=2)

# Add a 3 day lag with a custom column name on the open price
ds.add_lag(period=3, col_name='three_day_lag', on='open').dropna().head()

,timestamp,symbol,series,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,isin,lag_close_1,lag_close_2,three_day_lag,chg_close_2,rol_mean_close_3
36,2018-09-06,FEDERALBNK,EQ,78.50,78.50,76.70,77.35,77.30,77.75,6731974,5.218088e+08,30898,INE171A01029,77.75,77.00,81.50,-0.037152,77.366667
37,2018-09-06,HDFCBANK,EQ,2049.00,2059.00,2032.60,2052.20,2050.15,2045.85,2600603,5.316363e+09,125530,INE040A01026,2045.85,2051.80,2069.40,-0.014072,2049.950000
38,2018-09-06,ICICIBANK,EQ,330.00,331.25,325.50,328.65,327.95,329.65,11322771,3.724325e+09,81857,INE090A01021,329.65,328.50,343.60,-0.013467,328.933333
39,2018-09-06,IDFCBANK,EQ,45.15,45.40,44.45,44.90,44.70,44.85,12803268,5.735607e+08,23485,INE092T01019,44.85,44.95,47.95,-0.063674,44.900000
40,2018-09-06,AXISBANK,EQ,638.90,643.30,624.60,638.20,637.50,637.65,10026621,6.344121e+09,122891,INE238A01034,637.65,641.80,655.45,0.009259,639.216667


You could use 
```python 
ds.data.info()
```
to see the added columns at the end of each cell

## ``add_pct_change``
add a percentage change column

In [7]:
## Add a 2 day percentage change on close price
ds.add_pct_change(on='close', period=2)

,timestamp,symbol,series,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,isin,lag_close_1,lag_close_2,three_day_lag,chg_close_2
0,2018-09-03,AXISBANK,EQ,655.45,655.45,629.60,631.80,630.70,649.25,6494484,4.148248e+09,88364,INE238A01034,NaN,NaN,NaN,NaN
1,2018-09-03,RBLBANK,EQ,630.10,643.50,622.00,623.90,624.40,627.25,1532438,9.708929e+08,28755,INE976G01028,NaN,NaN,NaN,NaN
2,2018-09-03,INDUSINDBK,EQ,1906.00,1918.85,1884.70,1897.00,1895.00,1906.60,937981,1.782402e+09,85683,INE095A01012,NaN,NaN,NaN,NaN
3,2018-09-03,KOTAKBANK,EQ,1292.00,1295.00,1265.00,1269.10,1266.25,1287.25,2076747,2.639495e+09,59594,INE237A01028,NaN,NaN,NaN,NaN
4,2018-09-03,BANKBARODA,EQ,153.95,156.50,150.90,151.70,151.30,152.95,16081701,2.484263e+09,72339,INE028A01039,NaN,NaN,NaN,NaN
5,2018-09-03,SBIN,EQ,312.50,312.50,304.80,306.35,305.65,309.60,14708076,4.560945e+09,94708,INE062A01020,NaN,NaN,NaN,NaN
6,2018-09-03,PNB,EQ,88.30,89.30,86.00,86.35,86.00,88.15,30610408,2.695346e+09,76964,INE160A01022,NaN,NaN,NaN,NaN
7,2018-09-03,IDFCBANK,EQ,47.95,48.85,47.60,47.90,47.95,47.60,6484207,3.130970e+08,15826,INE092T01019,NaN,NaN,NaN,NaN
8,2018-09-03,ICICIBANK,EQ,343.60,344.00,332.55,334.15,335.00,342.60,12859450,4.339500e+09,88616,INE090A01021,NaN,NaN,NaN,NaN
9,2018-09-03,HDFCBANK,EQ,2069.40,2078.95,2063.50,2075.05,2073.95,2061.20,1915402,3.973617e+09,65178,INE040A01026,NaN,NaN,NaN,NaN


Calculate the 2 day percentage change in close price and lag it by one day.
This is especially useful if you want to know the 2 day returns on the morning of the 3rd day

In [8]:
ds.add_pct_change(on='close', period=2, lag=1)
ds.data.head()


,timestamp,symbol,series,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,isin,lag_close_1,lag_close_2,three_day_lag,chg_close_2
0,2018-09-03,AXISBANK,EQ,655.45,655.45,629.6,631.8,630.70,649.25,6494484,4.148248e+09,88364,INE238A01034,NaN,NaN,NaN,NaN
1,2018-09-03,RBLBANK,EQ,630.10,643.50,622.0,623.9,624.40,627.25,1532438,9.708929e+08,28755,INE976G01028,NaN,NaN,NaN,NaN
2,2018-09-03,INDUSINDBK,EQ,1906.00,1918.85,1884.7,1897.0,1895.00,1906.60,937981,1.782402e+09,85683,INE095A01012,NaN,NaN,NaN,NaN
3,2018-09-03,KOTAKBANK,EQ,1292.00,1295.00,1265.0,1269.1,1266.25,1287.25,2076747,2.639495e+09,59594,INE237A01028,NaN,NaN,NaN,NaN
4,2018-09-03,BANKBARODA,EQ,153.95,156.50,150.9,151.7,151.30,152.95,16081701,2.484263e+09,72339,INE028A01039,NaN,NaN,NaN,NaN


You could also pass keyword arguments to the percent change function. These arguments are passed on to the pandas function before computing the results.
So let's compute the percent change by backfilling NA data

In [9]:
ds.add_pct_change(on='close', period=2, lag=1, fill_method='bfill');

## ``add_rolling``
add a rolling statistic column

In [10]:
ds.add_rolling(window=3, on='close', function='mean');

mean


## ``add_formula``
add a formula column

The formula should be a string

In [11]:
ds.add_formula()

TypeError: add_formula() missing 2 required positional arguments: 'formula' and 'col_name'

## ``add_indicator``

add a technical indicator

In [ ]:
ds.add_indicator('RSI', 2);

## A few shortcuts

Adding columns in bulk.

In [ ]:
# Add 2,3,5 day returns
[ds.add_pct_change(on='close', period=i, col_name='ret' + str(i))
 for i in [2,3,5]]
ds.data.info()

## ``batch_process``
Batch process